## An example workflow

In [ ]:
#to use the full width of the browser window uncomment the code below and execute the cell
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
from pyPoseidon.model import *
from pyPoseidon.dem import *
from pyPoseidon.meteo import *
import datetime
from pyPoseidon.utils.data import *

In [ ]:
import holoviews as hv
import geoviews as gv
import geoviews.feature as gf
from cartopy import crs


hv.notebook_extension('bokeh')
#hv.notebook_extension('matplotlib')

### Define case

In [ ]:
#define in a dictionary the properties of the model..
dic={'minlon':-35., # lat/lon window
     'maxlon':42.,
     'minlat':25.05,
     'maxlat':76.5,
     'solver':'d3d',
     'tag':'eur025',
     'resolution':0.25, #grid resoltuion 
     'step':60, # step for output of map field in d3d
     'rstep': 60*12 ,  # step for restart file
     'start_date':'2010-2-1',
     'time_frame':'12H',
     'ft1':0, # initial forecasting hour per file
     'ft2':11, # final forecasting hour per file
#     'meteo':'hnms_oper',
     'meteo':'ecmwf_oper',
     'dem': 'gebco',
     'dpath' : '/Users/brey/DATA/GEBCO_2014_2D.nc',
     'epath':'/Users/brey/DELFT3D/SVN/7545/bin/lnx64/', #folder for solver executables
     'ncores': 4 , #number of cores
     'rpath':'/Users/brey/Downloads/EUR1/20100201.00/', #location of calc folder
     'Dt':1.0, # add any additional relevant setup variable,
     'conda_env': 'mpi3', # optional conda env for the solver
     'update':['meteo']
    }

#### Local operational ECMWF files

In [ ]:
if 'time_frame' in dic.keys(): end_date= pd.to_datetime(dic['start_date']) + pd.to_timedelta(dic['time_frame'])
dic.update({'end_date':end_date.strftime(format='%Y-%m-%d %H:%M:%S')})

dr = pd.date_range(dic['start_date'],dic['end_date'], freq='12H')

#creating a sequence of folder from which we read the meteo.
PATH='/Users/brey/Downloads/'#Path to meteo files
folders = [datetime.datetime.strftime(x, '%Y%m%d.%H') for x in dr]
meteo = [PATH+'{:04d}/{:02d}/{:02d}/'.format(x.year,x.month,x.day)+datetime.datetime.strftime(x, '%Y%m%d.%H')+'.tropical_cyclone.grib' for x in dr]
meteo

#### Local operational HNMS files

In [ ]:
if 'time_frame' in dic.keys(): end_date= pd.to_datetime(dic['start_date']) + pd.to_timedelta(dic['time_frame'])
dic.update({'end_date':end_date.strftime(format='%Y-%m-%d')})

dr = pd.date_range(dic['start_date'],dic['end_date'], freq='12H')
dr

In [ ]:
meteo = []
#creating a sequence of folder from which we read the meteo.
PATH='/Users/brey/Downloads/HNMS/01/'
files = glob.glob(PATH+'/E_JRC00*')
files.sort()
meteo= meteo + files[:12]

files = glob.glob(PATH+'/E_JRC12*')
files.sort()
meteo = meteo + files[:13]
meteo

#### Local operational AM files

In [ ]:
if 'time_frame' in dic.keys(): end_date= pd.to_datetime(dic['start_date']) + pd.to_timedelta(dic['time_frame'])
dic.update({'end_date':end_date.strftime(format='%Y-%m-%d')})

dr = pd.date_range(dic['start_date'],dic['end_date'], freq='12H')
dr

In [ ]:
meteo = []
#creating a sequence of folder from which we read the meteo.
PATH='/Users/brey/Downloads/AM'
files = glob.glob(PATH+'/JRC_*_00_*')
files.sort()
meteo= meteo + files[:12]

files = glob.glob(PATH+'/JRC_*_12_*')
files.sort()
meteo = meteo + files[:13]
meteo

#### update dictionary

In [ ]:
dic.update({'mpaths':meteo})

## Initialize

In [ ]:
#initialize a model
b = model(**dic)

### set it up

In [ ]:
b.set(**dic) #set it up 

#### Optional adjust the wet area based on a coastline shapefile

In [ ]:
b.impl.dem.impl.adjust('/Users/brey/DATA/COASTLINES/naturalearth/coastline/ne_50m_coastline.shp',nc=50) 

In [ ]:
b.impl.dem.impl.dem.fval.plot()# test coverage

## Save to folder for execution 

In [ ]:
#set the run by saving the files
b.output()

In [ ]:
# save model info for further use
b.save()

In [ ]:
# save all matrices for further use
#b.pickle(path=path)

### execute

In [ ]:
#execute
b.run()

## Analysis of output 

In [ ]:
%matplotlib inline

In [ ]:
otp = data([dic['rpath']])#,tag='eur025')# optional tag for data selection ,sa_date='2010-2-1 5:0:0',se_date='2010-2-1 22:0:0') # optionally set range for tide gauge data acquisition 

In [ ]:
otp.vars

In [ ]:
otp.vars.time

In [ ]:
otp.dem.bathymetry.plot()

In [ ]:
fig = plt.figure(figsize=(12,8))
ax = plt.subplot(projection=ccrs.PlateCarree());

otp.dem.bathymetry.plot.pcolormesh('longitude', 'latitude', ax=ax);

gp = ax.scatter(otp.x, otp.y, s=2, color='k',  label='grid points', transform=ccrs.PlateCarree());

pp = ax.scatter(otp.xh, otp.yh, s=2, color='b', label='pressure points', transform=ccrs.PlateCarree());

ax.legend()

ax.coastlines('50m'); ax.gridlines(draw_labels=True);

### Visualize

In [ ]:
%%opts Image [width=650 height=450] (cmap='jet')
otp.hview('S1').to(hv.Image, ['XZ','YZ'], 'S1').hist()

In [ ]:
g50 = gv.feature.coastline(plot=dict(scale='50m'), style=dict(linewidth=1.5))

In [ ]:
%%opts Image [colorbar=True width=650 height=450 toolbar="above", tools=['hover']] (cmap='viridis')
otp.gview('S1').to(gv.Image, ['XZ','YZ'],dynamic=True) * g50 * gf.borders()

In [ ]:
otp.frames(['S1'],title='SSH',vmin=-.5, vmax=.5, nv=20)

In [ ]:
otp.frames(['U1Z','V1Z'],title='Vel',scale=.1)

## Observation points / Validation

In [ ]:
otp.obs.locations

In [ ]:
# Get with index number
p = otp.obs.iloc(4)
print p.head()
print p.tail()

In [ ]:
#get with Station Name  
p = otp.obs.loc('Denmark - Hirtshals')
p.head()

In [ ]:
#plot
ax = p.plot()

### sample nearest point from simulation 

In [ ]:
 plat,plon = otp.obs.locations.loc[16,['lat','lon']]

In [ ]:
#ts = otp.vars.S1.sel(XZ=[lon], method='nearest', tolerance=5).sel(YZ=[lat], method='nearest', tolerance=5)
#ts = ts.squeeze().to_pandas()
#ts.head()

In [ ]:
otp.info

In [ ]:
ts = point(lon=plon,lat=plat,data=otp)
ts.tseries(var='S1')

In [ ]:
ts.S1

In [ ]:
ts.S1.plot()

In [ ]:
## Join the graphs
ax = ts.S1.plot(figsize=(14,10),color=['r'],label='d3d')
p.plot(ax=ax)
ax.legend(loc='center left',bbox_to_anchor=(1.1, 0.5))